In [4]:
%load_ext autoreload
%autoreload 2
import traceback
import time
from processingmm import batch_processing_win7, batch_processing
from processingmm.helpers import load_filenames_raw_data, add_path, load_path_win7
from processingmm.batch_processing import find_processed_folders, find_all_folders, create_folders_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
import shutil 

chunk_size = 2
max_nb = 1

data = load_path_win7()

In [6]:
data

{'base_directory': 'F:\\test_dev\\test',
 'backup': 'F:\\test_dev\\backup',
 'base_temp_directory': 'F:\\test_dev\\test_temp',
 'calib_directory': 'F:\\polarimetry\\calibration',
 'directory_computation': 'F:\\test_dev\\temp',
 'folder_NAS': 'D:\\DataRaw'}

In [7]:
# 1. Backup the data in the B: drive
base_directory = [data['base_directory']]
backup = [data['backup']]
base_temp_directory = data['base_temp_directory']
calib_directory = data['calib_directory']
directory_computation = data['directory_computation']
folder_NAS = data['folder_NAS']

In [8]:
def batch_process_win7(base_directory: list, backup: list, base_temp_directory: str, calib_directory: str, 
                       directory_computation: str):
    batch_processing_win7.backup_to_B_drive(base_directory, backup)
    
    processed = []
    
    df = batch_processing.get_df_processing(base_directory)
    to_process = batch_processing.get_to_process(df, run_all = False)[0:chunk_size]
    _ = batch_processing.move_folders_temp(base_directory, base_temp_directory, to_process)
    
    while len(to_process) > 0:
        try:
            shutil.rmtree(directory_computation)
        except FileNotFoundError:
            pass
        except:
            traceback.print_exc()
        try:
            os.mkdir(directory_computation)
        except FileExistsError:
            pass
        except:
            traceback.print_exc()

        to_process = batch_processing.batch_process([base_temp_directory], calib_directory, run_all = False, parameter_set = 'CUSA', 
                                                    max_nb = 1, target_temp = directory_computation)
            
        _ = batch_processing.move_folders_temp(directory_computation, base_temp_directory, to_process, put_back = True)

        try:
            shutil.rmtree(directory_computation)
        except:
            traceback.print_exc()

        df = batch_processing.get_df_processing([base_temp_directory])
        to_process = batch_processing.get_to_process(df, run_all = False)

    processed = batch_processing_win7.move_back_the_folders(base_directory, base_temp_directory)
    
    # 5. Move the data to the NAS
    batch_processing_win7.move_to_NAS(base_directory, processed, folder_NAS)

In [10]:
finished = True
while finished:
    batch_process_win7(base_directory, backup, base_temp_directory, calib_directory, directory_computation)
    time.sleep(25)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:27<00:00, 27.84s/it]


KeyboardInterrupt: 